In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 03/12, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Scoring Breakdown
Question | Points
--- | ---
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 2
3bi	| 1
3bii |	2
3ci	| 1
3cii	| 2
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
8ai | 0
8aii | 1
8aiii | 1
8b | 1
8c | 1
8d | 2
8e | 1
**Total** | 72

In [1]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [2]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

In [3]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -c -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [4]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [5]:
%%sql
SELECT * FROM halloffame LIMIT 5

Run the following cell for grading purposes.

In [6]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **not** use any sub-queries.

In [7]:
%%sql result_1a <<
...

In [8]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [ ]:
grader.check("q1a")

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans (specifically, looking at the execution time, cost, and overall query plan structure). 

__Note__: To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

See the subsequent cells for an example of how to display the query plan.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the execution time and cost for each query.

In [11]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

In [12]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

In [13]:
inducted_hof_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

In [14]:
provided_query_cost = ...
provided_query_timing = ...
your_query_cost = ...
your_query_timing = ...

In [ ]:
grader.check("q1bi")

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [17]:
q1b_part2 = ...

In [ ]:
grader.check("q1bii")

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [20]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
...
;
SELECT * FROM inducted_hof_ca;

In [21]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [ ]:
grader.check("q1c")

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [24]:
%%sql with_view <<
...

In [25]:
%%sql without_view <<
...

In [26]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [ ]:
grader.check("q1di")

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [32]:
with_view_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

In [33]:
without_view_query_str = ...
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

In [34]:
with_view_cost = ...
with_view_timing = ...
without_view_cost = ...
without_view_timing = ...

In [ ]:
grader.check("q1dii")

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [38]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
...
;
SELECT * FROM inducted_hof_ca_mat;

In [39]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [ ]:
grader.check("q1ei")

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [42]:
%%sql with_materialized_view <<
...

In [43]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [ ]:
grader.check("q1eii")

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [46]:
with_materialized_view_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

In [47]:
with_materialized_view_cost = ...
with_materialized_view_timing = ...

In [ ]:
grader.check("q1eiii")

#### Question 1eiv:
Given your findings from inspecting the query plans of queries using a view, no view, and a materialized view, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [50]:
q1e_part4 = ...

In [ ]:
grader.check("q1eiv")

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your findings and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [53]:
query_view_str = ...
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

In [54]:
query_view_cost = ...
query_view_timing = ...

In [ ]:
grader.check("q2a")

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [57]:
%%sql result_2b <<
...

In [58]:
query_view_with_filter_str = ...
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

In [59]:
query_view_with_filter_cost = ...
query_view_with_filter_timing = ...

In [ ]:
grader.check("q2b")

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [62]:
q2c = ...

In [ ]:
grader.check("q2c")

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your findings and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 3: Joins
### Question 3a:
Perform a left, right, inner, and full join on the `people` and `collegeplaying` tables on the `playerid` column and inspect their respective query plans (in particular, look at the number of rows resulting from the query).

In [65]:
%%sql left_join <<
...

In [66]:
left_join_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $left_join_query_str"

In [67]:
%%sql right_join <<
...

In [68]:
right_join_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $right_join_query_str"

In [69]:
%%sql inner_join <<
...

In [70]:
inner_join_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $inner_join_query_str"

In [71]:
%%sql full_join <<
...

In [72]:
full_join_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $full_join_query_str"

In [73]:
# Do not delete/edit this cell
left_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_left.csv', index=False)
right_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_right.csv', index=False)
inner_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_inner.csv', index=False)
full_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_full.csv', index=False)

In [ ]:
grader.check("q3a")

### Question 3b:
#### Question 3bi:
Given your findings from inspecting the query plan of the different joins, answer the following question. Assign the variable `q3b_part1` to a list of all the options which are true for the proposed statement.

1. For the query joining `people` and `collegeplaying` tables on the `playerid`,  the `INNER JOIN` output is the same as:
    1. the `LEFT JOIN` output
    1. the `RIGHT JOIN` output
    1. the `FULL JOIN` output
    1. None of the above 

**Note:** Your answer should be formatted as follows: `q3b_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3bi
points: 1
-->

In [76]:
q3b_part1 = ...

In [ ]:
grader.check("q3bi")

<!-- BEGIN QUESTION -->

#### Question 3bii
Explain your answer to the previous part. If you did not select answer choice `D`, ensure your explanation discusses if your findings can be generalized for all queries.

<!--
BEGIN QUESTION
name: q3bii
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

#### Question 3ci:
Given your findings from inspecting the query plan of the different joins, answer the following question. Assign the variable `q3c_part1` to a list of all options which are true for the proposed statement.

1. For the query joining `people` and `collegeplaying` tables on the `playerid`, the `FULL JOIN` output is the same as:
    1. the `LEFT JOIN` output
    1. the `RIGHT JOIN` output
    1. the `INNER JOIN` output
    1. None of the above   

**Note:** Your answer should be formatted as follows: `q3c_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3ci
points: 1
-->

In [79]:
q3c_part1 = ...

In [ ]:
grader.check("q3ci")

<!-- BEGIN QUESTION -->

#### Question 3cii

Explain your answer to the previous part. If you did not select answer choice `D`, ensure your explanation discusses if your findings can be generalized for all queries.

<!--
BEGIN QUESTION
name: q3cii
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 4: Indexes

#### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Inspect the query plan and record the execution time and cost.

In [82]:
%%sql result_4a <<
...

In [83]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [ ]:
grader.check("q4ai")

In [86]:
result_4a_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

In [87]:
result_4a_cost = ...
result_4a_timing = ...

In [ ]:
grader.check("q4aii")

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [90]:
%%sql result_4b <<
...

Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [91]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

In [92]:
result_4b_cost = ...
result_4b_timing = ...

In [ ]:
grader.check("q4b")

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, drop the index before moving onto the next question.

In [95]:
%%sql 
...

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [96]:
%%sql result_4c <<
...

Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [97]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

In [98]:
result_4c_cost = ...
result_4c_timing = ...

In [ ]:
grader.check("q4c")

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [101]:
q4d_part1 = ...

In [ ]:
grader.check("q4di")

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your findings from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [104]:
%%sql 
...

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games. Your query should join the `salaries` and `appearances` table on `yearid`, `teamid`, and `playerid`.

In [105]:
%%sql result_4ei <<
...

In [106]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [ ]:
grader.check("q4ei_part1")

Inspect the query plan and record the execution time and cost.

In [109]:
result_4ei_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

In [110]:
result_4ei_cost = ...
result_4ei_timing = ...

In [ ]:
grader.check("q4ei_part2")

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [112]:
%%sql result_4eii << 
...

In [113]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

In [114]:
result_4eii_with_index_cost = ...
result_4eii_with_index_timing = ...

In [ ]:
grader.check("q4eii")

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [117]:
%%sql 
...

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on `yearid`, `teamid`, and `playerid`.

In [118]:
%%sql result_4eiii <<
...

In [119]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [ ]:
grader.check("q4eiii_part1")

In [122]:
result_4eiii_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

In [123]:
result_4eiii_cost = ...
result_4eiii_timing = ...

In [ ]:
grader.check("q4eiii_part2")

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [125]:
%%sql result_4eiv << 
...

In [126]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

In [127]:
result_4eiv_with_index_cost = ...
result_4eiv_with_index_timing = ...

In [ ]:
grader.check("q4eiv")

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [130]:
%%sql 
...

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [131]:
%%sql result_4v <<
...

In [132]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

In [133]:
result_4ev_multiple_col_index_cost = ...
result_4ev_multiple_col_index_timing = ...

In [ ]:
grader.check("q4ev")

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [136]:
q4e_part6 = ...

In [ ]:
grader.check("q4evi")

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 5:
#### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [139]:
%%sql result_5a_min << 
...

In [140]:
result_5a_min_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

In [141]:
result_5a_min_query_cost = ...
result_5a_min_query_timing = ...

In [ ]:
grader.check("q5ai")

In [143]:
%%sql result_5a_avg <<
...

In [144]:
result_5a_avg_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

In [145]:
result_5a_avg_query_cost = ...
result_5a_avg_query_timing = ...

In [ ]:
grader.check("q5aii")

#### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [148]:
%%sql 
...

In [149]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

In [150]:
result_5b_min_query_cost = ...
result_5b_min_query_timing = ...

In [ ]:
grader.check("q5bi")

In [153]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

In [154]:
result_5b_avg_query_cost = ...
result_5b_avg_query_timing = ...

In [ ]:
grader.check("q5bii")

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [157]:
%%sql 
...

#### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [158]:
q5c = ...

In [ ]:
grader.check("q5c")

<!-- BEGIN QUESTION -->

#### Question 5d:

Explain your answer to the previous part based on your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 6
In this question, we will inspect the impact of that clustering our data on an index can have on a query's performance.

#### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [161]:
%%sql result_6a <<
...

In [162]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [ ]:
grader.check("q6ai")

In [165]:
result_6a_query_str = ...
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [166]:
result_6a_cost = ...
result_6a_timing = ...

In [ ]:
grader.check("q6aii")

#### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [169]:
%sql \di

In [170]:
%%sql 
...

In [171]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [172]:
result_6b_cost = ...
result_6b_timing = ...

In [ ]:
grader.check("q6b")

#### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table and cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [175]:
%%sql 
...

In [176]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

In [177]:
result_6c_cost = ...
result_6c_timing = ...

In [ ]:
grader.check("q6c")

#### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [180]:
q6d = ...

In [ ]:
grader.check("q6d")

<!-- BEGIN QUESTION -->

#### Question 6e:

Explain your answer to the previous part based on your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 7
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

#### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [183]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

Next, run the provided update script and record the wall time.

In [184]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100000..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [185]:
result_7a_timing = ...

In [ ]:
grader.check("q7a")

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [187]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

#### Question 7b:
Now, create an index on the `salary` column and record the wall time after executing the update script.

In [188]:
%%sql 
...

In [189]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100000..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [190]:
result_7b_timing = ...

In [ ]:
grader.check("q7b")

## Question 8
In this question, we will explore the benefits of bulk loading data and indexes. 

#### Question 8ai:
Create a new table called `fabricated_salaries` with the following schema:

Column Name | Data Type
--- | --- 
yearid |  INTEGER
teamid | CHARACTER VARYING(3)
lgid | CHARACTER VARYING(2)
playerid | CHARACTER VARYING(10)
salary | DOUBLE PRECISION

**Note:** There are no tests associated with this subpart.

In [192]:
%%sql result_8a <<
...

#### Question 8aii:
Record the time it takes to copy the contents of the `fabricated_salaries.txt` into the new table. Hint: Use the psql shell comamnd `\copy` to perform the copying (see the documentation [here](https://www.postgresql.org/docs/9.2/app-psql.html#APP-PSQL-META-COMMANDS-COPY)). Your answer should be formatted as follows: `!psql -h localhost -d baseball -c "\copy _REPLACE_ME_"`.

In [193]:
# Unzip the file before copying
!unzip -u data/salaries_trunc.zip -d data/

In [194]:
%time
...

In [195]:
time_to_copy_data = ...

In [ ]:
grader.check("q8aii")

#### Question 8aiii:
Record the time to create an index on the `salary` column with name `fabricated_salary_idx` for the new table you created.

In [197]:
%%time
%%sql 
...

In [198]:
time_to_create_index = ...

In [ ]:
grader.check("q8aiii")

### Question 8b:
Now, create a second table called `fabricated_salaries2` with the same schema definition from `Question 8ai`. Then, record the time to run 10,000 `INSERT` statements.

In [200]:
%%sql result_8b <<
...

Run the following cell and record the time it takes to perform 10,000 random inserts into the table you just created.

In [201]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..110000 LOOP
     INSERT INTO fabricated_salaries2 (yearid, teamid, lgid, playerid, salary)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000);
     END LOOP;
END;
$$;

In [202]:
time_8b = ...

In [ ]:
grader.check("q8b")

### Question 8c
Now, create a third table `fabricated_salaries3` (with the same schema specified in `Question 8ai`) and create an index on the `salary` column with name `fabricated_salary3_idx`.

In [204]:
%%sql result_8c <<
...

Create an index on the `salary` column of the new table you created.

In [205]:
%%sql 
...

Run the following cell and record the time it takes to perform 10,000 random inserts into the table you just created.

In [206]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..110000 LOOP
     INSERT INTO fabricated_salaries3 (yearid, teamid, lgid, playerid, salary)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000);
     END LOOP;
 END;
$$;

In [207]:
time_8c = ...

In [ ]:
grader.check("q8c")

### Question 8d
Now, write an update query that adds $5,000 to every salary entry in the table you created in `Question 8ai`. Record the time it takes to execute the query.

In [209]:
%%time
%%sql
...

In [210]:
time_8d = ...

In [ ]:
grader.check("q8d")

## Question 8e
Let's compare the time from the previous part with the time it takes to drop the index, run the same update query from `Question 8c`, and then recreate the index.

In [212]:
%%time
%%sql
BEGIN;
...
COMMIT

In [213]:
time_8e = ...

In [ ]:
grader.check("q8e")

## Congratulations! You have finished Project 2.

Run the following cell to zip the results of your queries. You will also need to run the export cell at the end of the notebook. **For submission on Gradescope, you will need to submit both the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

In [215]:
!zip -r results.zip results

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()